In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,lib.cnmem=0.7,floatX=float32'

In [2]:
import theano, numpy
import theano.tensor as T
import time
import sys
from collections import defaultdict

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [3]:
data_path = '../data/'
train = pickle.load(open(data_path + 'train_feature_myadding1_only', 'rb'))
print 'loading done'

loading done


In [4]:
data_path = '../data/'
test = pickle.load( open(data_path + 'test_feature_myadding1_only', 'rb'))
print 'loading done'

loading done


In [5]:
data_path = '../data/'
user_ID_map = pickle.load( open(data_path + 'user_ID_map', 'rb'))
songs_ID_map = pickle.load( open(data_path + 'songs_ID_map', 'rb'))
print 'loading done'
print train.head()
num_users = len(user_ID_map)
num_items = len(songs_ID_map)
print num_users, num_items

loading done
    msno  song_id  target
0  20479   135901       1
1   4676   257686       1
2   4676   273457       1
3   4676    49669       1
4  20479    39092       1
34403 584719


In [6]:
class BPR_Item(object):

    def __init__(self, rank, n_users, n_items, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._n_users = n_users
        self._n_items = n_items
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._learning_rate = learning_rate
        self._configure_theano()
        self._generate_train_model_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        """
          Configures Theano to run in fast mode
          and using 32-bit floats. 
        """
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'

    def _generate_train_model_function(self):
        u = T.lvector('u')
        i = T.lvector('i')
        j = T.lvector('j')

        h = numpy.random.random((self._n_items, self._rank))
        b = numpy.random.random(self._n_items)
        
        self.W = theano.shared(numpy.random.random((self._n_users, self._rank)).astype('float32'), name='W')
        self.H = theano.shared(h.astype('float32'), name='H')
        self.B = theano.shared(b.astype('float32'), name='B')
        
        x_ui = T.dot(self.W[u], self.H[i].T).diagonal() + self.B[i]
        x_uj = T.dot(self.W[u], self.H[j].T).diagonal() + self.B[j]
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.W[u] ** 2).sum(axis=1) - 
                    self._lambda_i * (self.H[i] ** 2).sum(axis=1) - self._lambda_j * 
                    (self.H[j] ** 2).sum(axis=1) - self._lambda_bias * 
                    (self.B[i] ** 2 + self.B[j] ** 2) )
       
    
        cost = - obj

        g_cost_W = T.grad(cost=cost, wrt=self.W)
        g_cost_H = T.grad(cost=cost, wrt=self.H)
        g_cost_B = T.grad(cost=cost, wrt=self.B)

        updates = [(self.W, self.W - self._learning_rate * g_cost_W), (self.H, self.H - self._learning_rate * g_cost_H), 
                   (self.B, self.B - self._learning_rate * g_cost_B) ]

        self.train_model = theano.function(inputs=[u, i, j], outputs=cost, updates=updates)

    
    def train(self, train1=train, batch_size=1000):
        """
          Trains the BPR Matrix Factorisation model using Stochastic
          Gradient Descent and minibatches over `train_data`.
          `train_data` is an array of (user_index, item_index) tuples.
          We first create a set of random samples from `train_data` for 
          training, of size `epochs` * size of `train_data`.
          We then iterate through the resulting training samples by
          batches of length `batch_size`, and run one iteration of gradient
          descent for the batch.
        """
        if len(train1) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train)))
            batch_size = len(train1)
        
        z = 0
        n_sgd_samples = len(train1)
        t2 = t1 = t0 = time.time()
        k = 5
        for k in range(k):
            while (z+1)*batch_size < len(train1):
                self.train_model(
                    train1[z*batch_size: (z+1)*batch_size].msno.tolist(),
                    train1[z*batch_size: (z+1)*batch_size].song_id_x.tolist(),
                    train1[z*batch_size: (z+1)*batch_size].song_id_y.tolist()
                )
                z += 1
                t2 = time.time()
                sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
                sys.stderr.flush()
                t1 = t2
        if len(train1) > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/len(train1)))
            sys.stderr.flush()
            
    def _generate_test_model_function(self):
        """
          Computes item predictions for `user_index`.
          Returns an array of prediction values for each item
          in the dataset.
        """
        u = T.lvector('u')
        i = T.lvector('i')
        j = T.lvector('j')

        x_ui = T.dot(self.W[u], self.H[i].T).diagonal() + self.B[i]
        x_uj = T.dot(self.W[u], self.H[j].T).diagonal() + self.B[j]
        x_uij = x_ui-x_uj
        
        self.test_model = theano.function(inputs=[u, i, j], outputs=x_uij)
   
    def test_bundle(self, train = train, batch_size=1000):
        """
          Computes the Area Under Curve (AUC) on `test_data`.
          `test_data` is an array of (user_index, item_index) tuples.
          During this computation we ignore users and items
          that didn't appear in the training data, to allow
          for non-overlapping training and testing sets.
        """
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(train)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                train[z*batch_size: (z+1)*batch_size].msno.tolist(),
                train[z*batch_size: (z+1)*batch_size].song_id_x.tolist(),
                train[z*batch_size: (z+1)*batch_size].song_id_y.tolist()
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [7]:
# cols = ['msno', 'song_id', 'target']
# train = train[cols]

In [8]:
# test = test[['msno', 'song_id']]

In [9]:
# pickle.dump(test,  open(data_path + 'test_feature_myadding1_only', 'wb'))
# print 'OK'
# pickle.dump(train,  open(data_path + 'train_feature_myadding1_only', 'wb'))

In [10]:
length = int(len(train)*0.8)
length = int(len(train)*0.2)
trainSet = train[length:]
#trainSet = train[:length]
#valSet = train[length:]

In [11]:
# print len(trainSet), len(valSet)

In [12]:
train_pos = trainSet[trainSet.target == 1][['msno', 'song_id']]
train_neg = trainSet[trainSet.target == 0][['msno', 'song_id']]
train_pos["msno"] = train_pos["msno"].astype('int')
train_neg["msno"] = train_neg["msno"].astype('int')
print train_pos.head()
train = train_pos.merge(train_neg[['msno', 'song_id']]  , on='msno', how='inner')
print train.head()
print train.dtypes

          msno  song_id
1475483  17258   227855
1475484  17258    76790
1475485  21634   257890
1475486  16505   355344
1475487  16505    18739
    msno  song_id_x  song_id_y
0  17258     227855     240153
1  17258     227855     118860
2  17258     227855     253196
3  17258     227855     221229
4  17258     227855      55706
msno         int64
song_id_x    int64
song_id_y    int64
dtype: object


In [13]:
# val_pos = valSet[valSet.target == 1][['msno', 'song_id']]
# val_neg = valSet[valSet.target == 0][['msno', 'song_id']]
# val_pos["msno"] = val_pos["msno"].astype('int')
# val_neg["msno"] = val_neg["msno"].astype('int')
# print val_pos.head()
# val = val_pos.merge(val_neg[['msno', 'song_id']]  , on='msno', how='inner')
# print val.head()
# print val.dtypes

In [14]:
print len(train)
sample = np.random.choice(range(len(train)), len(train)//10, replace = False)
#train1 = pd.DataFrame(np.random.choice(range(len(train)), len(train) // 100, replace = False), columns=train.columns)

712702773


In [15]:
train1 = train.iloc[sample]

In [16]:
# print len(train1)
# print len(val)

In [17]:
bpr_item = BPR_Item(50, num_users, num_items)

In [18]:
# bpr_item.train(train1)

Processed 71270000 ( 100.00% ) in 0.1104 seconds
Total training time 8462.72 seconds; 1.187412e-04 per sample


In [22]:
# bpr_item.test_bundle(train1)

Current AUC mean (71270000 samples): 0.95294onds


0.95294315981478872

In [23]:
# data_path = '../data/'
# pickle.dump(bpr_item, open(data_path + 'bpr_item', 'wb'))

In [42]:
H_item = np.matrix(bpr_item.H.eval())
B_item = np.matrix(bpr_item.B.eval()).T
W_item = np.matrix(bpr_item.W.eval())

In [43]:
print np.shape(B_item), np.shape(W_item), np.shape(H_item)

(584719, 1) (34403, 50) (584719, 50)


In [49]:
train_pos['user_weight'] = train_pos.msno.apply(lambda x : W_item[x])

In [50]:
train_pos['item_weight'] = train_pos.song_id.apply(lambda x : H_item[x])

In [51]:
train_pos['item_bias'] = train_pos.song_id.apply(lambda x : B_item[x])

In [53]:
train_pos['score'] = train_pos.apply(lambda x: (B_item[x['song_id']] + H_item[x['song_id']]\
                                                * W_item[x['msno']].T)[0,0], axis = 1)

In [54]:
print 'done'

done


In [55]:
train_neg['score'] = train_neg.apply(lambda x: (B_item[x['song_id']] + H_item[x['song_id']]\
                                                * W_item[x['msno']].T)[0,0], axis = 1)

In [65]:
train_pos1 = train_pos.drop(['user_weight', 'item_weight', 'item_bias'], axis = 1)

In [66]:
print train_pos1.dtypes

msno         int64
song_id      int64
score      float64
dtype: object


In [67]:
data_path = '../data/'
pickle.dump(train_pos1, open(data_path + 'train_pos_bpr', 'wb'))
print 'done'
pickle.dump(train_neg, open(data_path + 'train_neg_bpr', 'wb'))


done


In [69]:
for user in train_neg.msno.unique():
    if user in user_limit: continue
    score = np.max(train_neg[train_neg.msno == user]['score'])
    user_limit[user] = score

In [68]:
user_limit = {}
for user in train_pos.msno.unique():
    pos = np.min(train_pos[train_pos.msno == user]['score'])
    user_limit[user] = pos

In [74]:
test['score'] = test.apply(lambda x: (B_item[x['song_id']] + H_item[x['song_id']]\
                                                * W_item[x['msno']].T)[0,0], axis = 1)
# test.msno.apply(lambda x: user_limit[x] if x in user_limit else 0.5)

In [79]:
print test.dtypes

msno         int64
song_id      int64
target     float64
score      float64
dtype: object


In [78]:
test["msno"] = test["msno"].astype('int')

In [88]:
def get_target(row):
    if row['msno'] not in user_limit:
        return 0.5
    return 1 if row['score'] > user_limit[row['msno']] else 0

In [86]:
test['target'] = test.apply(lambda x: get_target(x), axis = 1)

In [89]:
test['target']  = test['target'].apply(lambda x: 1 if True else False)

In [72]:
data_path = '../data/'
#test = pickle.load( open(data_path + 'test_feature_myadding', 'rb'))
test1 = pickle.load( open(data_path + 'test_feature_myadding1_reduceDim', 'rb'))
print 'loading done'

loading done


In [90]:
print test['target'].head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [92]:
print ('Saving predictions Model model of gbdt')
subm = pd.DataFrame()
subm['id'] = test1['id'].values
subm['target'] = test['target'].values
subm.to_csv(data_path + '../submissions/submission_bpr.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')

print('Done!')

Saving predictions Model model of gbdt
Done!


In [26]:
#bpr_item.test_bundle(val)
#2196

In [ ]:
#20
#0.81675onds
#0.54
#//100
#50
#0.88
#0.55

#//50
#50
#0.91
#0.55

#//10
#50
#0.95
#0.546

#//all train //10
#0.95294onds